# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
"""
Set your required experiment parameters
"""
#LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
LEARNING_RATE = [1e-4, 1e-5]
OPTIMIZERS = ['SGD','RMSprop','Adagrad','Adam'] 
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95


In [ ]:
results = {}
"""
建立你的訓練與實驗迴圈並蒐集資料
"""
def runOptimizer(opt, lr, MOMENTUM):
    if (opt=='SGD'):
        optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=MOMENTUM)
    elif (opt=='RMSprop'):
        optimizer = keras.optimizers.RMSprop(lr=lr, epsilon=None, decay=0.0)
    elif (opt=='Adagrad'):
        optimizer = keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=0.0)
    elif (opt=='Adam'):
        optimizer = keras.optimizers.Adam(lr=lr, epsilon=None, decay=0.0)
    return optimizer

for lr in LEARNING_RATE:
    for opt in OPTIMIZERS:
        keras.backend.clear_session() # 把舊的 Graph 清掉
        print("Experiment with LR = %.6f in %s optimizer" % (lr, opt))
        model = build_mlp(input_shape=x_train.shape[1:])
        model.summary()
        optimizer = runOptimizer(opt, lr, MOMENTUM)
        model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

        model.fit(x_train, y_train, 
                  epochs=EPOCHS, 
                  batch_size=BATCH_SIZE, 
                  validation_data=(x_test, y_test), 
                  shuffle=True)

        # Collect results
        train_loss = model.history.history["loss"]
        valid_loss = model.history.history["val_loss"]
        train_acc = model.history.history["acc"]
        valid_acc = model.history.history["val_acc"]

        exp_name_tag = "exp-lr-%s-opt-%s" % (str(lr), opt)
        results[exp_name_tag] = {'train-loss': train_loss,
                                 'valid-loss': valid_loss,
                                 'train-acc': train_acc,
                                 'valid-acc': valid_acc}

Experiment with LR = 0.000100 in SGD optimizer
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [===========

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 8s 168us/step - loss: 1.9552 - acc: 0.2953 - val_loss: 1.8177 - val_acc: 0.3442
Epoch 2/50
50000/50000 [==============================] - 8s 155us/step - loss: 1.7845 - acc: 0.3671 - val_loss: 1.8757 - val_acc: 0.3397
Epoch 3/50
50000/50000 [==============================] - 8s 156us/step - loss: 1.7030 - acc: 0.3968 - val_loss: 1.6963 - val_acc: 0.3928
Epoch 4/50
50000/50000 [==============================] - 8s 155us/step - loss: 1.6456 - acc: 0.4174 - val_loss: 1.6360 - val_acc: 0.4238
Epoch 5/50
50000/50000 [==============================] - 8s 155us/step - loss: 1.6026 - acc: 0.4326 - val_loss: 1.7001 - val_acc: 0.3929
Epoch 6/50
50000/50000 [==============================] - 8s 155us/step - loss: 1.5718 - acc: 0.4453 - val_loss: 1.6455 - val_acc: 0.4219
Epoch 7/50
50000/50000 [==============================] - 8s 155us/step - loss: 1.5381 - acc: 0.4552 - val_loss: 1.5742 - 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
將實驗結果繪出
"""
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()